In [1]:
import os
import cv2
import base64
from ultralytics import YOLO
from IPython.display import HTML
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
# Load YOLOv8 model
model = YOLO('yolov8n.pt')

In [3]:
file_1 = "/Users/nikhil20.sharma/Library/CloudStorage/OneDrive-RelianceCorporateITParkLimited/xxxWork/2-VideoInteligence/Data/officeMovie.mp4"
file_2 = "/Users/nikhil20.sharma/Library/CloudStorage/OneDrive-RelianceCorporateITParkLimited/xxxWork/2-VideoInteligence/Data/officeRecording.mp4"
file_3 = "/Users/nikhil20.sharma/Library/CloudStorage/OneDrive-RelianceCorporateITParkLimited/xxxWork/2-VideoInteligence/Data/8972/8972_C9.mp4"
file_4 = "/Users/nikhil20.sharma/Library/CloudStorage/OneDrive-RelianceCorporateITParkLimited/xxxWork/2-VideoInteligence/Data/8972_C9_2m_chuncks/chunk_5.mp4"
file_5 = "/Users/nikhil20.sharma/Library/CloudStorage/OneDrive-RelianceCorporateITParkLimited/xxxWork/2-VideoInteligence/Data/8910/8910_C6.mp4"
file_6 = "/Users/nikhil20.sharma/Library/CloudStorage/OneDrive-RelianceCorporateITParkLimited/xxxWork/2-VideoInteligence/8910_C6.mp4"

source = file_4

In [4]:
# ffmpeg -i /Users/nikhil20.sharma/Library/CloudStorage/OneDrive-RelianceCorporateITParkLimited/xxxWork/2-VideoInteligence/Data/8910/8910_C6.mp4 -c:v libx264 8910_C6.mp4

In [5]:
output_file_name = 'output_' + os.path.basename(source)
output_file_name

'output_chunk_5.mp4'

In [ ]:
results = model(
    source=source,
    visualize=False,  # For Debugging purposes
    # save_crop=True,
    stream=True,
    classes=[0]
)

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename=output_file_name)  # save to disk

In [6]:
cap = cv2.VideoCapture("./Data/officeRecording.mp4")

In [7]:
cap.isOpened()

True

In [8]:
success, frame = cap.read()
success

True

In [ ]:
[x for x in dir(cap) if '__' not in x]

In [6]:
source = './Data/RawFromBusiness/processed_8667_C2.mp4'
source = './data/officeMovie.mp4'

In [7]:
def check_video_file(video_input_path: str=None):
    """Check if the video file is valid and readable."""

    if video_input_path is None:
        raise ValueError(f"Please provide a valid video input path.\n\t- {video_input_path}")
    
    if not os.path.isfile(video_input_path):
        raise ValueError(f"The provided video input path is not found.\n\t- {video_input_path}")

    # Open the input video file
    cap = cv2.VideoCapture(video_input_path)
    
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_input_path}")
        return
    
    # Get video properties
    frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    codec = "".join([chr((fourcc >> 8 * i) & 0xFF) for i in range(4)])
    duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)

    print(f'''
[+] Video properties:
    - duration    : {round(duration,2)} seconds
    - frame width : {frame_width}
    - frame height: {frame_height}
    - frame count : {frame_count}
    - fps         : {fps}
    - codec       : {codec}
''')

In [20]:
# def main(model='yolov8n.pt'):
# """This function will handel all other function and call them based on the USECASE"""

# Load the YOLOv8 model
model = YOLO(model='yolov8n.pt')
print(f'[+] Loaded the model [{model}]')

# Check video file properties
check_video_file(video_input_path=source)

# Open the video file
cap = cv2.VideoCapture(source)
if not cap.isOpened():
    print(f"Error: Unable to open video file {source}")

error_count = 0
frame_count = 0

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    frame_count += 1

    if not success:
        error_count += 1
        print(f"Error: Could not read frame. ({error_count})")
        continue
    
    # Display the frame
    # cv2.imshow('Video Playback', frame)

    if success:
        
        # Display the frame
        # cv2.imshow('Video Playback', frame)

        # Run YOLOv8 inference on the frame
        # results = model(frame)
        results = model(
            source=frame,
            visualize=False,  # For Debugging purposes
            # save_crop=True,
            stream=False,
            classes=[0]
        )



        # Process the results
        person_count = 0
        for result in results:
            if hasattr(result, 'boxes'):
                boxes = result.boxes
                for box in boxes:
                    class_id = box.cls.item()  # Class ID
                    if class_id == 0:  # Assuming '0' is the class ID for 'person'
                        person_count += 1

        # Draw the count of people at the top right corner of the frame
        text = f"People count: {person_count}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_thickness = 2
        text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        text_x = frame.shape[1] - text_size[0] - 10
        text_y = text_size[1] + 10

        cv2.putText(frame, text, (text_x, text_y), font, font_scale, (0, 255, 0), font_thickness)








        # Visualize the results on the frame
        frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()




# main()


[+] Loaded the model [YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine

In [19]:
help(YOLO)

Help on class YOLO in module ultralytics.models.yolo.model:

class YOLO(ultralytics.engine.model.Model)
 |  YOLO(model='yolov8n.pt', task=None, verbose=False)
 |  
 |  YOLO (You Only Look Once) object detection model.
 |  
 |  Method resolution order:
 |      YOLO
 |      ultralytics.engine.model.Model
 |      torch.nn.modules.module.Module
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model='yolov8n.pt', task=None, verbose=False)
 |      Initialize YOLO model, switching to YOLOWorld if model filename contains '-world'.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  task_map
 |      Map head to model, trainer, validator, and predictor classes.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __annotations__ = {}
 |  
 |  ----------------------------------------------------------------------
 

In [11]:
source

'./Data/RawFromBusiness/processed_8667_C2.mp4'

In [16]:
results

NameError: name 'results' is not defined